In [31]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

In [21]:
df = pd.read_csv("./Tweets.csv")

df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [22]:
df.groupby(["airline_sentiment"]).size()

airline_sentiment
negative    9178
neutral     3099
positive    2363
dtype: int64

In [23]:
df = df[df["airline_sentiment_confidence"] >= 0.8]

df.groupby(["airline_sentiment"]).size()

airline_sentiment
negative    7392
neutral     1550
positive    1517
dtype: int64

In [24]:
token = Tokenizer(num_words=100)

token.fit_on_texts(df["text"].values)

In [25]:
X = token.texts_to_sequences(df["text"].values)
X = pad_sequences(X, padding="post", maxlen=100)

In [26]:
X

array([[97, 62,  0, ...,  0,  0,  0],
       [97, 99,  1, ...,  0,  0,  0],
       [97,  9, 99, ...,  0,  0,  0],
       ...,
       [13, 98, 93, ...,  0,  0,  0],
       [13, 89,  1, ...,  0,  0,  0],
       [13,  6, 23, ...,  0,  0,  0]], dtype=int32)

In [27]:
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df["airline_sentiment"])

y

array([1, 0, 0, ..., 0, 1, 0])

In [28]:
y = to_categorical(y)

y

array([[0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]], dtype=float32)

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [35]:
model = Sequential()

model.add(Embedding(input_dim=len(token.word_index), output_dim=128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0, activation="tanh", recurrent_activation="sigmoid", unroll=False, use_bias=True, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(64, dropout=0.2, recurrent_dropout=0, activation="tanh", recurrent_activation="sigmoid", unroll=False, use_bias=True))
model.add(Dense(3, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 100, 128)          1638656   
                                                                 
 spatial_dropout1d_6 (Spati  (None, 100, 128)          0         
 alDropout1D)                                                    
                                                                 
 lstm_7 (LSTM)               (None, 100, 128)          131584    
                                                                 
 dropout_2 (Dropout)         (None, 100, 128)          0         
                                                                 
 lstm_8 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 3)                 195       
                                                      

In [36]:
model.fit(X_train, y_train, epochs=100, batch_size=30, verbose=True, validation_data=(X_test, y_test))

Epoch 1/100
245/245 [==============================] - 50s 193ms/step - loss: 0.8245 - accuracy: 0.7000 - val_loss: 0.7884 - val_accuracy: 0.7192
Epoch 2/100
245/245 [==============================] - 47s 190ms/step - loss: 0.8193 - accuracy: 0.7014 - val_loss: 0.7910 - val_accuracy: 0.7192
Epoch 3/100
245/245 [==============================] - 46s 188ms/step - loss: 0.8189 - accuracy: 0.7014 - val_loss: 0.7944 - val_accuracy: 0.7192
Epoch 4/100
245/245 [==============================] - 48s 195ms/step - loss: 0.8189 - accuracy: 0.7014 - val_loss: 0.7937 - val_accuracy: 0.7192
Epoch 5/100
245/245 [==============================] - 48s 194ms/step - loss: 0.8180 - accuracy: 0.7014 - val_loss: 0.7888 - val_accuracy: 0.7192
Epoch 6/100
245/245 [==============================] - 47s 190ms/step - loss: 0.8183 - accuracy: 0.7014 - val_loss: 0.7889 - val_accuracy: 0.7192
Epoch 7/100
245/245 [==============================] - 47s 194ms/step - loss: 0.8178 - accuracy: 0.7014 - val_loss: 0.7904 -

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)

print(loss)
print(accuracy)